In [1]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from haversine import haversine, Unit, haversine_vector
from datetime import datetime
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import os
from dotenv import load_dotenv
import datashader as ds
import datashader.transfer_functions as tf
from datashader.utils import lnglat_to_meters
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from causalnex.structure.notears import from_pandas
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE
from causalnex.structure import StructureModel
from sklearn.model_selection import train_test_split
import networkx as nx

In [2]:
df1=pd.read_csv("/home/moraa/Documents/10_academy/Week-8/data/driver_locations_during_request.csv")

In [3]:
df2=pd.read_csv("/home/moraa/Documents/10_academy/Week-8/data/nb.csv")

In [4]:
# Drop the specified columns
df1.drop(columns=['created_at', 'updated_at'], inplace=True)

# Display the DataFrame after dropping the columns
df1.head()

,id,order_id,driver_id,driver_action,lat,lng
0,1,392001,243828,accepted,6.602207,3.270465
1,2,392001,243588,rejected,6.592097,3.287445
2,3,392001,243830,rejected,6.596133,3.281784
3,4,392001,243539,rejected,6.596142,3.280526
4,5,392001,171653,rejected,6.609232,3.288800


In [5]:
df1.head()

,id,order_id,driver_id,driver_action,lat,lng
0,1,392001,243828,accepted,6.602207,3.270465
1,2,392001,243588,rejected,6.592097,3.287445
2,3,392001,243830,rejected,6.596133,3.281784
3,4,392001,243539,rejected,6.596142,3.280526
4,5,392001,171653,rejected,6.609232,3.288800


# Data Exploration

In [6]:
import pandas as pd

# Impute missing values in Trip Start Time and Trip End Time with mode
df2 = df2.copy()
df2['Trip Start Time'] = df2['Trip Start Time'].fillna(df2['Trip Start Time'].mode()[0])
df2['Trip End Time'] = df2['Trip End Time'].fillna(df2['Trip End Time'].mode()[0])

# Display the DataFrame after imputation
df2.head()

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36


# Feature Engineering

Create Day of the Week Feature

In [7]:
# Ensure Trip Start Time is in datetime format
df2['Trip Start Time'] = pd.to_datetime(df2['Trip Start Time'])

# Extract day of the week
df2['Day of Week'] = df2['Trip Start Time'].dt.day_name()

# Display the first few rows to verify
df2[['Trip Start Time', 'Day of Week']].head()


,Trip Start Time,Day of Week
0,2021-07-01 07:28:04,Thursday
1,2021-07-01 06:38:04,Thursday
2,2021-07-01 06:21:02,Thursday
3,2021-07-01 07:16:07,Thursday
4,2021-07-01 09:30:59,Thursday


In [8]:
df2['Day of Week'].nunique()

7

Create Time of Day Category

In [9]:
# Function to categorize time of day
def categorize_time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

# Extract hour from Trip Start Time
df2['Hour'] = df2['Trip Start Time'].dt.hour

# Apply the function to create time category
df2['Time of Day'] = df2['Hour'].apply(categorize_time_of_day)

# Display the first few rows to verify
df2[['Trip Start Time', 'Hour', 'Time of Day']].head()


,Trip Start Time,Hour,Time of Day
0,2021-07-01 07:28:04,7,Morning
1,2021-07-01 06:38:04,6,Morning
2,2021-07-01 06:21:02,6,Morning
3,2021-07-01 07:16:07,7,Morning
4,2021-07-01 09:30:59,9,Morning


In [10]:
df2['Time of Day'].nunique()

4

Creating Holiday

In [11]:
# Define a list of public holidays in Nigeria for 2021 and 2022
holidays_2021 = [
    '2021-01-01', '2021-04-02', '2021-04-05', '2021-05-01', '2021-05-12', '2021-05-13',
    '2021-06-12', '2021-07-20', '2021-07-21', '2021-10-01', '2021-10-18', '2021-10-19',
    '2021-12-25', '2021-12-26'
]

holidays_2022 = [
    '2022-01-01', '2022-01-03', '2022-04-15', '2022-04-18', '2022-05-01', '2022-05-02',
    '2022-05-03', '2022-05-04', '2022-06-12', '2022-06-13', '2022-07-09', '2022-07-10',
    '2022-07-11', '2022-07-12', '2022-10-01', '2022-10-03', '2022-10-08', '2022-10-09',
    '2022-10-10', '2022-12-25', '2022-12-26', '2022-12-27'
]

# Combine the lists of holidays
all_holidays = pd.to_datetime(holidays_2021 + holidays_2022)

# Create a feature indicating if the date is a holiday
df2['Is Holiday'] = df2['Trip Start Time'].dt.date.isin(all_holidays.date)

df2['Is Holiday'].head()


0    False
1    False
2    False
3    False
4    False
Name: Is Holiday, dtype: bool

In [12]:
df2['Is Holiday'].nunique()

2

In [13]:
df1.dtypes

id                 int64
order_id           int64
driver_id          int64
driver_action     object
lat              float64
lng              float64
dtype: object

In [14]:
df2.head()

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,Day of Week,Hour,Time of Day,Is Holiday
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37,Thursday,7,Morning,False
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28,Thursday,6,Morning,False
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23,Thursday,6,Morning,False
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42,Thursday,7,Morning,False
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,Thursday,9,Morning,False


In [15]:
# Ensure Trip Start Time and Trip End Time are in datetime format
df2['Trip Start Time'] = pd.to_datetime(df2['Trip Start Time'])
df2['Trip End Time'] = pd.to_datetime(df2['Trip End Time'])

# Split Trip Origin and Trip Destination into latitude and longitude
df2[['Origin Lat', 'Origin Lng']] = df2['Trip Origin'].str.split(',', expand=True).astype(float)
df2[['Destination Lat', 'Destination Lng']] = df2['Trip Destination'].str.split(',', expand=True).astype(float)

# Extract additional time-based features
df2['Start Hour'] = df2['Trip Start Time'].dt.hour
df2['Start Day of Week'] = df2['Trip Start Time'].dt.dayofweek

# Calculate trip duration in minutes
df2['Trip Duration'] = (df2['Trip End Time'] - df2['Trip Start Time']).dt.total_seconds() / 60

# Rename columns to match for merging
df1.rename(columns={'order_id': 'Trip ID'}, inplace=True)

# Merge the datasets on 'Trip ID'
df_merged = pd.merge(df1, df2, on='Trip ID')

df_merged


,id,Trip ID,driver_id,driver_action,lat,lng,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,...,Hour,Time of Day,Is Holiday,Origin Lat,Origin Lng,Destination Lat,Destination Lng,Start Hour,Start Day of Week,Trip Duration
0,1,392001,243828,accepted,6.602207,3.270465,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,...,9,Morning,False,6.601042,3.276634,6.450107,3.391615,9,3,3.616667
1,2,392001,243588,rejected,6.592097,3.287445,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,...,9,Morning,False,6.601042,3.276634,6.450107,3.391615,9,3,3.616667
2,3,392001,243830,rejected,6.596133,3.281784,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,...,9,Morning,False,6.601042,3.276634,6.450107,3.391615,9,3,3.616667
3,4,392001,243539,rejected,6.596142,3.280526,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,...,9,Morning,False,6.601042,3.276634,6.450107,3.391615,9,3,3.616667
4,5,392001,171653,rejected,6.609232,3.288800,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,...,9,Morning,False,6.601042,3.276634,6.450107,3.391615,9,3,3.616667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1557735,1557736,517948,243774,rejected,6.469036,3.566877,"6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,...,11,Morning,False,6.443374,3.552312,6.500608,3.598194,11,4,102.783333
1557736,1557737,517948,245447,rejected,6.425431,3.527358,"6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,...,11,Morning,False,6.443374,3.552312,6.500608,3.598194,11,4,102.783333
1557737,1557738,517948,239866,rejected,6.440013,3.525378,"6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,...,11,Morning,False,6.443374,3.552312,6.500608,3.598194,11,4,102.783333
1557738,1557739,517948,243774,rejected,6.469036,3.566877,"6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,...,11,Morning,False,6.443374,3.552312,6.500608,3.598194,11,4,102.783333


In [16]:
df_merged.dtypes

id                            int64
Trip ID                       int64
driver_id                     int64
driver_action                object
lat                         float64
lng                         float64
Trip Origin                  object
Trip Destination             object
Trip Start Time      datetime64[ns]
Trip End Time        datetime64[ns]
Day of Week                  object
Hour                          int64
Time of Day                  object
Is Holiday                     bool
Origin Lat                  float64
Origin Lng                  float64
Destination Lat             float64
Destination Lng             float64
Start Hour                    int64
Start Day of Week             int64
Trip Duration               float64
dtype: object

Feature Extraction and Scaling

In [17]:
# Function to calculate distance between two geographical points
def calculate_distance(row, lat1, lon1, lat2, lon2):
    point1 = (row[lat1], row[lon1])
    point2 = (row[lat2], row[lon2])
    return geodesic(point1, point2).kilometers

# Calculate distance between driver location and trip origin
df_merged['Driver Distance to Origin'] = df_merged.apply(lambda row: calculate_distance(row, 'lat', 'lng', 'Origin Lat', 'Origin Lng'), axis=1)

# Calculate distance between trip origin and destination
df_merged['Trip Distance'] = df_merged.apply(lambda row: calculate_distance(row, 'Origin Lat', 'Origin Lng', 'Destination Lat', 'Destination Lng'), axis=1)




In [18]:
# Select relevant features for scaling
features_to_scale = ['lat', 'lng', 'Origin Lat', 'Origin Lng', 'Destination Lat', 'Destination Lng', 'Trip Duration', 'Driver Distance to Origin', 'Trip Distance']

# Initialize scaler (using StandardScaler for now, MinMaxScaler can be used alternatively)
scaler = StandardScaler()

# Fit and transform the selected features
df_merged[features_to_scale] = scaler.fit_transform(df_merged[features_to_scale])

# Convert categorical features to dummy/indicator variables
# df_merged = pd.get_dummies(df_merged, columns=['Time of Day', 'Start Day of Week'])

# Display the first few rows of the transformed dataframe
df_merged.head()

,id,Trip ID,driver_id,driver_action,lat,lng,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,...,Is Holiday,Origin Lat,Origin Lng,Destination Lat,Destination Lng,Start Hour,Start Day of Week,Trip Duration,Driver Distance to Origin,Trip Distance
0,1,392001,243828,accepted,1.103260,-2.032688,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,...,False,1.085202,-1.959701,-1.175806,0.107751,9,3,0.029672,-0.965534,1.976083
1,2,392001,243588,rejected,0.934718,-1.714323,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,...,False,1.085202,-1.959701,-1.175806,0.107751,9,3,0.029672,-0.228285,1.976083
2,3,392001,243830,rejected,1.002008,-1.820457,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,...,False,1.085202,-1.959701,-1.175806,0.107751,9,3,0.029672,-0.885988,1.976083
3,4,392001,243539,rejected,1.002145,-1.844041,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,...,False,1.085202,-1.959701,-1.175806,0.107751,9,3,0.029672,-0.967434,1.976083
4,5,392001,171653,rejected,1.220381,-1.688913,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,...,False,1.085202,-1.959701,-1.175806,0.107751,9,3,0.029672,-0.167964,1.976083


Computes Distances, driving speed, shortest Distances, driving route distances and other key variables

In [19]:
# Function to clean latitude values
def clean_latitude(lat):
    if lat < -90:
        return -90  # Replace latitude less than -90 with -90
    elif lat > 90:
        return 90   # Replace latitude greater than 90 with 90
    return lat

# Apply latitude cleaning to 'Origin Lat' and 'Destination Lat' columns
df_merged['Origin Lat'] = df_merged['Origin Lat'].apply(clean_latitude)
df_merged['Destination Lat'] = df_merged['Destination Lat'].apply(clean_latitude)

# Function to compute geodesic distance
def compute_geodesic_distance(row):
    lat1 = row['Origin Lat']
    lon1 = row['Origin Lng']
    lat2 = row['Destination Lat']
    lon2 = row['Destination Lng']
    coords_1 = (lat1, lon1)
    coords_2 = (lat2, lon2)
    return geodesic(coords_1, coords_2).kilometers

# Function to compute Haversine distance
def compute_haversine_distance(row):
    lat1 = row['Origin Lat']
    lon1 = row['Origin Lng']
    lat2 = row['Destination Lat']
    lon2 = row['Destination Lng']
    coords_1 = (lat1, lon1)
    coords_2 = (lat2, lon2)
    return haversine(coords_1, coords_2, unit=Unit.KILOMETERS)

# Function to compute driving speed
def compute_driving_speed(row):
    distance = row['Haversine Distance']
    duration = row['Trip Duration']
    return distance / (duration / 60)  # in km/h

# Compute geodesic distance for each row
df_merged['Geodesic Distance'] = df_merged.apply(compute_geodesic_distance, axis=1)

# Compute haversine distance for each row
df_merged['Haversine Distance'] = df_merged.apply(compute_haversine_distance, axis=1)

# Compute average speed for each row
df_merged['Average Speed'] = df_merged.apply(compute_driving_speed, axis=1)

# Display the updated DataFrame
df_merged


,id,Trip ID,driver_id,driver_action,lat,lng,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,...,Destination Lat,Destination Lng,Start Hour,Start Day of Week,Trip Duration,Driver Distance to Origin,Trip Distance,Geodesic Distance,Haversine Distance,Average Speed
0,1,392001,243828,accepted,1.103260,-2.032688,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,...,-1.175806,0.107751,9,3,0.029672,-0.965534,1.976083,339.802736,340.662877,688863.580397
1,2,392001,243588,rejected,0.934718,-1.714323,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,...,-1.175806,0.107751,9,3,0.029672,-0.228285,1.976083,339.802736,340.662877,688863.580397
2,3,392001,243830,rejected,1.002008,-1.820457,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,...,-1.175806,0.107751,9,3,0.029672,-0.885988,1.976083,339.802736,340.662877,688863.580397
3,4,392001,243539,rejected,1.002145,-1.844041,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,...,-1.175806,0.107751,9,3,0.029672,-0.967434,1.976083,339.802736,340.662877,688863.580397
4,5,392001,171653,rejected,1.220381,-1.688913,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,...,-1.175806,0.107751,9,3,0.029672,-0.167964,1.976083,339.802736,340.662877,688863.580397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1557735,1557736,517948,243774,rejected,-1.116936,3.524931,"6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,...,-0.485125,2.866731,11,4,0.040542,1.243480,-0.055373,123.644523,124.232935,183859.537177
1557736,1557737,517948,245447,rejected,-1.843909,2.783971,"6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,...,-0.485125,2.866731,11,4,0.040542,1.360644,-0.055373,123.644523,124.232935,183859.537177
1557737,1557738,517948,239866,rejected,-1.600805,2.746854,"6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,...,-0.485125,2.866731,11,4,0.040542,1.019207,-0.055373,123.644523,124.232935,183859.537177
1557738,1557739,517948,243774,rejected,-1.116936,3.524931,"6.443373599999999,3.5523123","6.500608199999999,3.5981937",2021-09-10 11:47:26,2021-09-10 13:30:13,...,-0.485125,2.866731,11,4,0.040542,1.243480,-0.055373,123.644523,124.232935,183859.537177


In [20]:
df_merged.dtypes

id                                    int64
Trip ID                               int64
driver_id                             int64
driver_action                        object
lat                                 float64
lng                                 float64
Trip Origin                          object
Trip Destination                     object
Trip Start Time              datetime64[ns]
Trip End Time                datetime64[ns]
Day of Week                          object
Hour                                  int64
Time of Day                          object
Is Holiday                             bool
Origin Lat                          float64
Origin Lng                          float64
Destination Lat                     float64
Destination Lng                     float64
Start Hour                            int64
Start Day of Week                     int64
Trip Duration                       float64
Driver Distance to Origin           float64
Trip Distance                   

Computes the number of riders and order in circles of 500m from accepted and unfulfilled orders

In [21]:
from scipy.spatial import KDTree

# Extract coordinates from the dataframes
rider_coords = df_merged[['lat', 'lng']].to_numpy()
order_coords = df_merged[df_merged['Trip ID'] == 392001][['Trip Origin']].apply(lambda x: list(map(float, x['Trip Origin'].split(','))), axis=1).tolist()

# Define the radius of the circle (500m)
radius = 0.5  # in kilometers

# Create a KDTree for riders
tree = KDTree(rider_coords)

# Function to count riders within 500m of an accepted order
def count_accepted_riders(order_coord):
    indices = tree.query_ball_point(order_coord, radius)
    return len(indices)

# Calculate number of riders within 500m for each accepted order
accepted_orders_coords = df_merged[df_merged['driver_action'] == 'accepted'][['lat', 'lng']].to_numpy()
accepted_riders_in_circle = sum(count_accepted_riders(coord) for coord in accepted_orders_coords)

# Print the results
print("Number of riders within 500m of accepted orders:", accepted_riders_in_circle)

Number of riders within 500m of accepted orders: 4287414784


In [22]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

# Assuming your DataFrame is named df_merged
# Adding more temporal features
df_merged['Trip Start Time'] = pd.to_datetime(df_merged['Trip Start Time'])
df_merged['Trip End Time'] = pd.to_datetime(df_merged['Trip End Time'])

# Time Since Last Trip for Driver
df_merged['Previous Trip End Time'] = df_merged.groupby('driver_id')['Trip End Time'].shift(1)
df_merged['Time Since Last Trip'] = (df_merged['Trip Start Time'] - df_merged['Previous Trip End Time']).dt.total_seconds() / 3600

# Trip Duration Categories
df_merged['Trip Duration Category'] = pd.cut(df_merged['Trip Duration'], bins=[0, 15, 30, 60, np.inf], labels=['Short', 'Medium', 'Long', 'Very Long'])

# Origin-Destination Popularity
df_merged['Origin-Destination'] = df_merged['Trip Origin'] + ' to ' + df_merged['Trip Destination']
origin_destination_counts = df_merged['Origin-Destination'].value_counts().reset_index()
origin_destination_counts.columns = ['Origin-Destination', 'Origin-Destination Count']
df_merged = df_merged.merge(origin_destination_counts, on='Origin-Destination', how='left')

# Distance from City Center
city_center = (0.0, 0.0)  # Replace with actual city center coordinates
df_merged['Origin Distance to City Center'] = df_merged.apply(lambda row: geodesic((row['Origin Lat'], row['Origin Lng']), city_center).km, axis=1)
df_merged['Destination Distance to City Center'] = df_merged.apply(lambda row: geodesic((row['Destination Lat'], row['Destination Lng']), city_center).km, axis=1)

# Driver Acceptance Rate
driver_acceptance_rate = df_merged.groupby('driver_id')['driver_action'].apply(lambda x: (x == 'accepted').mean()).reset_index().rename(columns={'driver_action': 'Acceptance Rate'})
df_merged = df_merged.merge(driver_acceptance_rate, on='driver_id', how='left')

# Driver Experience
driver_experience = df_merged.groupby('driver_id').size().reset_index().rename(columns={0: 'Driver Experience'})
df_merged = df_merged.merge(driver_experience, on='driver_id', how='left')

# Driver's Average Trip Distance
driver_avg_trip_distance = df_merged.groupby('driver_id')['Trip Distance'].mean().reset_index().rename(columns={'Trip Distance': 'Avg Trip Distance'})
df_merged = df_merged.merge(driver_avg_trip_distance, on='driver_id', how='left')

# Driver's Average Trip Duration
driver_avg_trip_duration = df_merged.groupby('driver_id')['Trip Duration'].mean().reset_index().rename(columns={'Trip Duration': 'Avg Trip Duration'})
df_merged = df_merged.merge(driver_avg_trip_duration, on='driver_id', how='left')

# Driver's Average Speed
driver_avg_speed = df_merged.groupby('driver_id')['Average Speed'].mean().reset_index().rename(columns={'Average Speed': 'Avg Speed'})
df_merged = df_merged.merge(driver_avg_speed, on='driver_id', how='left')

# Drop potential infinite or NaN values resulting from division
df_merged.replace([np.inf, -np.inf], np.nan, inplace=True)
df_merged.dropna(inplace=True)

# Display the first few rows to check the new features
df_merged.head()



,id,Trip ID,driver_id,driver_action,lat,lng,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,...,Trip Duration Category,Origin-Destination,Origin-Destination Count,Origin Distance to City Center,Destination Distance to City Center,Acceptance Rate,Driver Experience,Avg Trip Distance,Avg Trip Duration,Avg Speed
16,17,392005,171165,rejected,0.467252,-0.165609,"6.565087699999999,3.3844415","6.499696300000001,3.3509075",2021-07-01 10:53:36,2021-07-01 11:27:51,...,Short,"6.565087699999999,3.3844415 to 6.4996963000000...",51,54.83083,73.36347,0.000875,5715,-0.027184,0.026039,197344.050231
17,18,392005,243172,rejected,0.573114,0.332253,"6.565087699999999,3.3844415","6.499696300000001,3.3509075",2021-07-01 10:53:36,2021-07-01 11:27:51,...,Short,"6.565087699999999,3.3844415 to 6.4996963000000...",51,54.83083,73.36347,0.051209,703,0.162084,0.035407,255134.934336
18,19,392005,244078,rejected,0.686601,-0.144470,"6.565087699999999,3.3844415","6.499696300000001,3.3509075",2021-07-01 10:53:36,2021-07-01 11:27:51,...,Short,"6.565087699999999,3.3844415 to 6.4996963000000...",51,54.83083,73.36347,0.010111,3066,-0.111655,-0.082415,199564.036556
20,21,392005,243392,rejected,0.502039,-0.232383,"6.565087699999999,3.3844415","6.499696300000001,3.3509075",2021-07-01 10:53:36,2021-07-01 11:27:51,...,Short,"6.565087699999999,3.3844415 to 6.4996963000000...",51,54.83083,73.36347,0.007194,3475,-0.076518,0.012019,192644.565309
22,23,392005,171165,rejected,0.467252,-0.165609,"6.565087699999999,3.3844415","6.499696300000001,3.3509075",2021-07-01 10:53:36,2021-07-01 11:27:51,...,Short,"6.565087699999999,3.3844415 to 6.4996963000000...",51,54.83083,73.36347,0.000875,5715,-0.027184,0.026039,197344.050231


In [23]:
import os
import pandas as pd

# Define the path to your artifacts folder
artifacts_folder = '/home/moraa/Documents/10_academy/Week-8/artifacts'

# Check if the artifacts folder exists; if not, create it
if not os.path.exists(artifacts_folder):
    os.makedirs(artifacts_folder)

# Specify the filename for your merged dataset
file_name = 'df_merged.csv'

# Define the full path to save the file
file_path = os.path.join(artifacts_folder, file_name)

# Save the DataFrame to CSV
df_merged.to_csv(file_path, index=False)

print(f"DataFrame 'df_merged' saved successfully to {file_path}")


DataFrame 'df_merged' saved successfully to /home/moraa/Documents/10_academy/Week-8/artifacts/df_merged.csv


In [24]:
df_merged.head()

,id,Trip ID,driver_id,driver_action,lat,lng,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,...,Trip Duration Category,Origin-Destination,Origin-Destination Count,Origin Distance to City Center,Destination Distance to City Center,Acceptance Rate,Driver Experience,Avg Trip Distance,Avg Trip Duration,Avg Speed
16,17,392005,171165,rejected,0.467252,-0.165609,"6.565087699999999,3.3844415","6.499696300000001,3.3509075",2021-07-01 10:53:36,2021-07-01 11:27:51,...,Short,"6.565087699999999,3.3844415 to 6.4996963000000...",51,54.83083,73.36347,0.000875,5715,-0.027184,0.026039,197344.050231
17,18,392005,243172,rejected,0.573114,0.332253,"6.565087699999999,3.3844415","6.499696300000001,3.3509075",2021-07-01 10:53:36,2021-07-01 11:27:51,...,Short,"6.565087699999999,3.3844415 to 6.4996963000000...",51,54.83083,73.36347,0.051209,703,0.162084,0.035407,255134.934336
18,19,392005,244078,rejected,0.686601,-0.144470,"6.565087699999999,3.3844415","6.499696300000001,3.3509075",2021-07-01 10:53:36,2021-07-01 11:27:51,...,Short,"6.565087699999999,3.3844415 to 6.4996963000000...",51,54.83083,73.36347,0.010111,3066,-0.111655,-0.082415,199564.036556
20,21,392005,243392,rejected,0.502039,-0.232383,"6.565087699999999,3.3844415","6.499696300000001,3.3509075",2021-07-01 10:53:36,2021-07-01 11:27:51,...,Short,"6.565087699999999,3.3844415 to 6.4996963000000...",51,54.83083,73.36347,0.007194,3475,-0.076518,0.012019,192644.565309
22,23,392005,171165,rejected,0.467252,-0.165609,"6.565087699999999,3.3844415","6.499696300000001,3.3509075",2021-07-01 10:53:36,2021-07-01 11:27:51,...,Short,"6.565087699999999,3.3844415 to 6.4996963000000...",51,54.83083,73.36347,0.000875,5715,-0.027184,0.026039,197344.050231
